In [16]:
%run ./common.ipynb
import logging


filepath='/Users/u1002018/Library/CloudStorage/OneDrive-SharedLibraries-FootLocker/Global Technology Services - DASH Doc Library/SOLE/'

#filepath='./output/QA/'


datestr = ""
partitionCnt=1


issueLists = [
          {'issueType':"Portfolio Initiative",'partitionCnt':1},
          {'issueType':"Product Initiative", 'partitionCnt':1 },
          {'issueType':"Epic",'partitionCnt':5},
          {'issueType':"Task",'partitionCnt':partitionCnt},
          {'issueType':"Sub-task",'partitionCnt':partitionCnt},
          {'issueType':"Bug",'partitionCnt':partitionCnt},
          {'issueType':"Incident",'partitionCnt':partitionCnt},
          {'issueType':"Production Defects",'partitionCnt':partitionCnt},
          {'issueType':"Defect",'partitionCnt':partitionCnt},
          {'issueType':"Issue",'partitionCnt':partitionCnt},
          {'issueType':"Test",'partitionCnt':partitionCnt},
          {'issueType':"Story",'partitionCnt':partitionCnt}
            ]
             
pd.set_option('display.max_columns', None)


In [17]:
def initDataframe():
    return pd.DataFrame (columns=['key','FixVersion', 
    'Priority',
    'Acceptance Criteria',
    'Severity Level',
    'Labels',
    'EpicLink',
    'Issuelinks',
    'Status',
    'Components',
    'issueType',
    'projectKey',
    'projectName',
    'created_ts',
    'updated_ts',
    'ParentLink',
    'summary'
    ])

In [18]:
global logger
logger = setup_logger(logging.INFO)
logger.info("=======================================")
logger.info("Script start accumulating data from JIRA")

2024-12-20 14:38:35,843 - root - INFO - =======================================
2024-12-20 14:38:35,844 - root - INFO - Script start accumulating data from JIRA


In [19]:
def data_clean(dataframe):
    if dataframe is not None and not dataframe.empty:
       dataframe= dataframe.filter(['key','fields.fixVersions', 
                                  'fields.priority.name',
                                  'fields.customfield_31601',
                                  'fields.customfield_12402.value',
                                  'fields.labels',
                                  'fields.customfield_10006',
                                  'fields.issuelinks',
                                  'fields.status.name',
                                  'fields.components',
                                  'fields.issuetype.name',
                                  'fields.project.key',
                                  'fields.project.name',
                                  'fields.created',
                                  'fields.updated',
                                  'fields.customfield_12823',
                                  'fields.summary']).rename(columns={'fields.fixVersions': 'FixVersion', 
                                  'fields.priority.name':'Priority',
                                  'fields.customfield_31601': 'Acceptance Criteria',
                                  'fields.customfield_12402.value':'Severity Level',
                                  'fields.labels':'Labels',
                                  'fields.customfield_10006':'EpicLink',
                                  'fields.issuelinks': 'Issuelinks',
                                  'fields.status.name': 'Status',
                                  'fields.components':'Components',
                                  'fields.issuetype.name':'issueType',
                                  'fields.project.key':'projectKey',
                                  'fields.project.name':'projectName',
                                  'fields.created':'created_ts',
                                  'fields.updated':'updated_ts',
                                  'fields.customfield_12823': 'ParentLink',
                                  'fields.summary':'summary',
                                                                    })
    return dataframe


In [20]:
def fetch( project, issueType):
    global fetch_df 
    logger.info(f"Fetch {issueType}...")


    attribute =' &expand=projects.issuetypes.fields'

    searchQry=f'?jql= project in ({project}) and issueType="{issueType}" and updated > startOfMonth(-11)'

    # if cond_flg == True: 
    #     searchQry=f"{searchQry} and status not in (Cancelled, Done)  {attribute}"
    # else:
    searchQry=f"{searchQry} {attribute}"

    fetch_df = getDataSet(searchQry)

    return data_clean(fetch_df)    
    

In [21]:
def split_list(lst, n):
    """Splits a list into n approximately equal parts."""
    k, m = divmod(len(lst), n)
    
    return [lst[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n)]

In [22]:
def convert_to_key_value(string):
    result = {}
   
    if string :
        pairs = string[0].split(',')  # Split string by comma
        # logger.info(pairs)
        idx =1
        for pair in pairs:
            key, value = pair.split('=')  # Split each pair by '='
            result[key.strip()] = value.strip()  # Add to dictionary, stripping whitespace
            idx = idx +1 
            if idx > 9:
                break
    return result

In [23]:
def createIssueLinkNode(issue,df,type,direction):
      dfi = pd.json_normalize(issue['fields'])
      # dft = pd.json_normalize(type)
      #   display(type)
      node = {'key': df['key'],'IssueLinkKey': issue['key'], 'projectKey' : issue['key'].split('-')[0],
            'status': dfi['status.name'][0], 'priority':dfi['priority.name'][0], 
            'issuetype':dfi['issuetype.name'][0], 
            'TypeName': type['name'], 'inward': type['inward'],'outward': type['outward'],'direction': direction}
      # display(node)
      return node

In [24]:
def getIssueLink(df):
    l_df = pd.DataFrame (columns=['key','IssueLinkKey', 
        'status',
        'priority',
        'issuetype',
        'inward',
        'outward',
        'projectKey'
        ])


    for index, row in df.iterrows():
        lst = row['Issuelinks']
        # itype = row['']

        if len(lst) > 0:
            idxlen = len(lst)
            for rec in lst:
                if rec.get("inwardIssue"):
                    inwardIssues = rec['inwardIssue']
                    l_df = l_df.append(createIssueLinkNode(inwardIssues,row,rec['type'],'inward'), ignore_index=True)
                if rec.get("outwardIssue"):
                    outwardIssues = rec['outwardIssue']
                    l_df = l_df.append(createIssueLinkNode(outwardIssues,row,rec['type'],'outward'), ignore_index=True)
    # display(l_df)
    return l_df
    # print (df['Issuelinks'].to_list()[0][idx]['inwardIssue']['fields'])

In [25]:
# Get all .csv files in the 'data' directory
csv_files = Path(filepath).glob("*_QA_working.csv")
for file in csv_files:
    filename=file.name
    os.remove(os.path.join(filepath, filename))

strings ='"SOLMerch"'

# strings = projectList()

for issueList in issueLists:
    # split_string_list = split_list(strings, issueList['partitionCnt'])
    # filtered_list = [series for series in split_string_list if not series.empty]
    # for partition in filtered_list:
        # logger.warning(f"processing %s - %s",partition,issueList['issueType'])

    df=fetch('SOLMerch',issueList['issueType'])
    if df is not None and not df.empty:
        g_Df = initDataframe()
        g_Df = pd.concat([g_Df, df ], ignore_index=True)
        g_Df=g_Df.fillna("NA")

        g_Df['FixVersion']= g_Df['FixVersion'].apply(lambda x: keyvalue(x,'name') if x is not None else "" )

        # display(g_Df)
        # g_Df['Issuelinks']= g_Df.apply(lambda row: getIssueLink(row['Issuelinks'], row['key']) )


        logger.info("Files uploaded")
        logger.info("Upload file to sharepoint...")

        try:
            if issueList['issueType'] in ['Portfolio Initiative']:
                output_path = os.path.join(filepath, 'Portfolio_Initiative_ParentEpic_QA_working.csv')
            else:              
                output_path = os.path.join(filepath, f"IssueLink_QA_working.csv")

            i_df = getIssueLink(g_Df)
            # display(i_df)
            if (len(i_df.index) > 0):
                i_df.to_csv(output_path ,mode='a', header=not os.path.exists(output_path), index=False)
        except Exception as e:  
            logger.error(f"Failed to upload files to {filepath}: {e}")
            pass

        # Delete the old DataFrame 
        # del(g_Df)

        # Perform garbage collection                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
        # gc.collect()
        
logger.info("Story Files uploaded")

logger.info("Move working files to current files")

# Get all .csv files in the 'data' directory
csv_files = Path(filepath).glob("*.csv")                                   
for file in csv_files:
    filename=file.name
    newname= filename.replace('_QA_working.csv','.csv')
    os.rename(os.path.join(filepath, filename), os.path.join(filepath, newname))
    logger.info(f"Script ended accumulating data from JIRA : {os.path.join(filepath, newname)}")
logger.info("=======================================")

2024-12-20 14:38:36,070 - root - INFO - Fetch Portfolio Initiative...
2024-12-20 14:38:36,071 - root - INFO - Getting report results...
2024-12-20 14:38:36,071 - root - INFO - https://jira.footlocker.com/rest/api/2/search?jql= project in (SOLMerch) and issueType="Portfolio Initiative" and updated > startOfMonth(-11)  &expand=projects.issuetypes.fields&maxResults=500&startAt=0
2024-12-20 14:38:36,677 - root - INFO - <Response [200]>
2024-12-20 14:38:36,677 - root - INFO - Report results received...
2024-12-20 14:38:36,678 - root - INFO - HTTP 200 - OK
2024-12-20 14:38:36,678 - root - INFO - total: 5
2024-12-20 14:38:36,703 - root - INFO - Files uploaded
2024-12-20 14:38:36,703 - root - INFO - Upload file to sharepoint...
2024-12-20 14:38:36,710 - root - INFO - Fetch Product Initiative...
2024-12-20 14:38:36,711 - root - INFO - Getting report results...
2024-12-20 14:38:36,711 - root - INFO - https://jira.footlocker.com/rest/api/2/search?jql= project in (SOLMerch) and issueType="Product 

In [26]:
# display(g_Df['Issuelinks'])

In [27]:
# g_Df[g_Df['Intial SOW']=='YES'][['Intial SOW','After Global Design','The Rudy Special','Baseline Scope']]

In [28]:
# len(g_Df[g_Df['key'] == 'SOLMERCH-3536']['Issuelinks'].to_list()[0][0])

In [29]:

# 15/* * * * *  /usr/bin/python3 /Users/u1002018/src/PAE/JIRA/py/sole_proj.py